In the name of Allah

In [232]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import time
from tqdm import tqdm
from fake_useragent import UserAgent

In [233]:
city = ["tehran", "isfahan", "mashhad", "urmia", "tabriz", "shiraz"]
sub_class = ["buy-apartment?has-photo=true", 
             "buy-office?has-photo=true", 
             "buy-villa?has-photo=true",
             "buy-old-house?has-photo=true", 
             "car?has-photo=true&exchange=exclude-exchanges", 
             "mobile-phones?has-photo=true&exchange=exclude-exchanges", 
             "laptop-notebook-macbook?has-photo=true&exchange=exclude-exchanges", 
             "motorcycles?has-photo=true&exchange=exclude-exchanges", 
             "refrigerator-freezer?has-photo=true&exchange=exclude-exchanges",
             "washing-machines?has-photo=true&exchange=exclude-exchanges"]

links = []
for s in sub_class:
    for c in city:
        links.append(f"https://divar.ir/s/{c}/{s}")

In [234]:
number_len = []
a_adv_urls = []

In [237]:
for url in tqdm(links[40:]):
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    data = soup.find_all("div", class_="post-card-item-af972 kt-col-6-bee95 kt-col-xxl-4-e9d46")
    number_len.append(len(data))
    for d in data:
        a = d.findChildren("a", recursive=False, href=True)[0]["href"]
        a_adv_urls.append([
            url.split("/")[-1].replace("?has-photo=true", "").replace("&exchange=exclude-exchanges", ""), 
            f"https://divar.ir{a}"])

100%|███████████████████████████████████████████████████████████████████████████████████| 20/20 [00:12<00:00,  1.67it/s]


In [238]:
f = open("./../data/urls.csv", "w")
for u in a_adv_urls:
    f.write(f"{u[0]}######{u[1]}\n")
f.close()

In [239]:
url_df = pd.read_csv("./../data/urls.csv", delimiter="######", header=None)
url_df

/var/folders/55/v51lw6zx0cx8px319jqb00hm0000gn/T/ipykernel_709/64299512.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  url_df = pd.read_csv("./../data/urls.csv", delimiter="######", header=None)


,0,1
0,buy-apartment,https://divar.ir/v/۱۲۵-متر-سه-خواب-نیاوران_آپا...
1,buy-apartment,https://divar.ir/v/ونک-۱۴۵-متر-۳-خواب-نوساز_آپ...
2,buy-apartment,https://divar.ir/v/۲۰۰-متر-طبقه-دهم-در-برج-مطر...
3,buy-apartment,https://divar.ir/v/33متر-بدون-خواب-فلت_آپارتما...
4,buy-apartment,https://divar.ir/v/90متر2خوابه-سعادت-اباد_آپار...
...,...,...
1435,washing-machines,https://divar.ir/v/ماشین-لباسشویی-سامسونگ_ماشی...
1436,washing-machines,https://divar.ir/v/ماشین-لباسشویی-سامسونگ_ماشی...
1437,washing-machines,https://divar.ir/v/ماشین-لباسشویی-توشیبا_ماشین...
1438,washing-machines,https://divar.ir/v/لباسشویی-زیرووات_ماشین-لباس...


## buy-apartment

In [291]:
def buy_apartment(soup):
    row_data = {"cat1": "real-estate", 
                "cat2": "buy-residential", 
                "cat3": "buy-apartment", 
                "title": soup.find_all("div", class_="kt-page-title__title kt-page-title__title--responsive-sized")[0].text, 
                "description": soup.find_all("p", class_="kt-description-row__text kt-description-row__text--primary")[0].text, 
                "url": url, 
                "sub_title": soup.find_all("div", class_="kt-page-title__subtitle kt-page-title__subtitle--responsive-sized")[0].text,
                "price": soup.find_all("p", class_="kt-unexpandable-row__value")[0].text.replace(" تومان", "").replace("(مقطوع)", "").replace("توافقی", "NAN"),  
                "size": soup.find_all("span", class_="kt-group-row-item__value")[0].text.replace(" متر", ""),
                "year": soup.find_all("span", class_="kt-group-row-item__value")[1].text.replace("قبل از ", ""), 
                "room_number": soup.find_all("span", class_="kt-group-row-item__value")[2].text.replace("بدون اتاق", "۰"), 
                "color": None, 
                "mileage": None,
                "CPU": None,
                "RAM": None
               }
    return row_data


def buy_office(soup):
    row_data = {"cat1": "real-estate", 
                "cat2": "buy-commercial-property", 
                "cat3": "buy-office", 
                "title": soup.find_all("div", class_="kt-page-title__title kt-page-title__title--responsive-sized")[0].text, 
                "description": soup.find_all("p", class_="kt-description-row__text kt-description-row__text--primary")[0].text, 
                "url": url, 
                "sub_title": soup.find_all("div", class_="kt-page-title__subtitle kt-page-title__subtitle--responsive-sized")[0].text,
                "price": soup.find_all("p", class_="kt-unexpandable-row__value")[0].text.replace(" تومان", "").replace("(مقطوع)", "").replace("توافقی", "NAN"),
                "size": soup.find_all("span", class_="kt-group-row-item__value")[0].text.replace(" متر", ""),
                "year": soup.find_all("span", class_="kt-group-row-item__value")[1].text.replace("قبل از ", ""), 
                "room_number": soup.find_all("span", class_="kt-group-row-item__value")[2].text.replace("بدون اتاق", "۰"), 
                "color": None, 
                "mileage": None,
                "CPU": None,
                "RAM": None
               }
    return row_data


def buy_villa(soup):
    row_data = {"cat1": "real-estate", 
                "cat2": "buy-residential", 
                "cat3": "buy-villa", 
                "title": soup.find_all("div", class_="kt-page-title__title kt-page-title__title--responsive-sized")[0].text, 
                "description": soup.find_all("p", class_="kt-description-row__text kt-description-row__text--primary")[0].text, 
                "url": url, 
                "sub_title": soup.find_all("div", class_="kt-page-title__subtitle kt-page-title__subtitle--responsive-sized")[0].text,
                "price": soup.find_all("p", class_="kt-unexpandable-row__value")[1].text.replace(" تومان", "").replace("(مقطوع)", "").replace("توافقی", "NAN"),
                "size": soup.find_all("p", class_="kt-unexpandable-row__value")[0].text.replace(" متر", ""),
                "year": soup.find_all("span", class_="kt-group-row-item__value")[1].text.replace("قبل از ", ""), 
                "room_number": soup.find_all("span", class_="kt-group-row-item__value")[2].text.replace("بدون اتاق", "۰"), 
                "color": None, 
                "mileage": None,
                "CPU": None,
                "RAM": None
               }
    return row_data


def buy_old_house(soup):
    row_data = {"cat1": "real-estate", 
                "cat2": "buy-residential", 
                "cat3": "buy-old-house", 
                "title": soup.find_all("div", class_="kt-page-title__title kt-page-title__title--responsive-sized")[0].text, 
                "description": soup.find_all("p", class_="kt-description-row__text kt-description-row__text--primary")[0].text, 
                "url": url, 
                "sub_title": soup.find_all("div", class_="kt-page-title__subtitle kt-page-title__subtitle--responsive-sized")[0].text,
                "price": soup.find_all("p", class_="kt-unexpandable-row__value")[1].text.replace(" تومان", "").replace("(مقطوع)", "").replace("توافقی", "NAN"),
                "size": soup.find_all("p", class_="kt-unexpandable-row__value")[0].text.replace(" متر", ""),
                "year": None, 
                "room_number": None, 
                "color": None, 
                "mileage": None,
                "CPU": None,
                "RAM": None
               }
    return row_data


def car(soup):
    row_data = {"cat1": "vehicles", 
                "cat2": "auto", 
                "cat3": "car", 
                "title": soup.find_all("div", class_="kt-page-title__title kt-page-title__title--responsive-sized")[0].text, 
                "description": soup.find_all("p", class_="kt-description-row__text kt-description-row__text--primary")[0].text, 
                "url": url, 
                "sub_title": soup.find_all("div", class_="kt-page-title__subtitle kt-page-title__subtitle--responsive-sized")[0].text,
                "price": soup.find_all("p", class_="kt-unexpandable-row__value")[-1].text.replace(" تومان", "").replace("(مقطوع)", "").replace("توافقی", "NAN"),
                "size": None,
                "year": soup.find_all("span", class_="kt-group-row-item__value")[1].text if soup.find_all("span", class_="kt-group-row-item__value") else None, 
                "room_number": None, 
                "color": soup.find_all("span", class_="kt-group-row-item__value")[2].text if len(soup.find_all("span", class_="kt-group-row-item__value")) > 2 else None, 
                "mileage": soup.find_all("span", class_="kt-group-row-item__value")[0].text if soup.find_all("span", class_="kt-group-row-item__value") else None,
                "CPU": None,
                "RAM": None
               }
    return row_data


def mobile_phones(soup):
    color = None
    for t in soup.find_all("p", class_="kt-base-row__title kt-unexpandable-row__title"):
        if "رنگ" in t.text:
            a = t.parent.parent
            color = a.find_all("p", class_="kt-unexpandable-row__value")[0].text

    ram = None
    for t in soup.find_all("p", class_="kt-base-row__title kt-unexpandable-row__title"):
        if "مقدار رم" in t.text:
            a = t.parent.parent
            ram = a.find_all("p", class_="kt-unexpandable-row__value")[0].text.replace(" گیگابایت", "")

    row_data = {"cat1": "electronic-devices", 
                "cat2": "mobile-tablet", 
                "cat3": "mobile-phones", 
                "title": soup.find_all("div", class_="kt-page-title__title kt-page-title__title--responsive-sized")[0].text, 
                "description": soup.find_all("p", class_="kt-description-row__text kt-description-row__text--primary")[0].text, 
                "url": url, 
                "sub_title": soup.find_all("div", class_="kt-page-title__subtitle kt-page-title__subtitle--responsive-sized")[0].text,
                "price": soup.find_all("p", class_="kt-unexpandable-row__value")[-1].text.replace(" تومان", "").replace("(مقطوع)", "").replace("توافقی", "NAN") if soup.find_all("p", class_="kt-unexpandable-row__value") else None,
                "size": None,
                "year": None, 
                "room_number": None, 
                "color": color, 
                "mileage": None,
                "CPU": None,
                "RAM": ram
               }
    return row_data


def laptop_notebook_macbook(soup):
    color = None
    for t in soup.find_all("p", class_="kt-base-row__title kt-unexpandable-row__title"):
        if "رنگ" in t.text:
            a = t.parent.parent
            color = a.find_all("p", class_="kt-unexpandable-row__value")[0].text

    cpu = None
    for t in soup.find_all("p", class_="kt-base-row__title kt-unexpandable-row__title"):
        if "پردازنده" in t.text:
            a = t.parent.parent
            cpu = a.find_all("p", class_="kt-unexpandable-row__value")[0].text

    ram = None
    for t in soup.find_all("p", class_="kt-base-row__title kt-unexpandable-row__title"):
        if "مقدار رم" in t.text:
            a = t.parent.parent
            ram = a.find_all("p", class_="kt-unexpandable-row__value")[0].text.replace(" گیگابایت", "")

    price = None
    for t in soup.find_all("p", class_="kt-base-row__title kt-unexpandable-row__title"):
        if "قیمت" in t.text:
            a = t.parent.parent
            price = a.find_all("p", class_="kt-unexpandable-row__value")[0].text.replace(" تومان", "").replace("(مقطوع)", "").replace("توافقی", "NAN")

    row_data = {"cat1": "electronic-devices", 
                "cat2": "computers", 
                "cat3": "laptop-notebook-macbook", 
                "title": soup.find_all("div", class_="kt-page-title__title kt-page-title__title--responsive-sized")[0].text, 
                "description": soup.find_all("p", class_="kt-description-row__text kt-description-row__text--primary")[0].text, 
                "url": url, 
                "sub_title": soup.find_all("div", class_="kt-page-title__subtitle kt-page-title__subtitle--responsive-sized")[0].text,
                "price": price,
                "size": None,
                "year": None, 
                "room_number": None, 
                "color": color, 
                "mileage": None,
                "CPU": cpu,
                "RAM": ram
               }
    return row_data


def motorcycles(soup):
    row_data = {"cat1": "vehicles", 
                "cat2": "motorcycles", 
                "cat3": None, 
                "title": soup.find_all("div", class_="kt-page-title__title kt-page-title__title--responsive-sized")[0].text, 
                "description": soup.find_all("p", class_="kt-description-row__text kt-description-row__text--primary")[0].text, 
                "url": url, 
                "sub_title": soup.find_all("div", class_="kt-page-title__subtitle kt-page-title__subtitle--responsive-sized")[0].text,
                "price": soup.find_all("p", class_="kt-unexpandable-row__value")[-1].text.replace(" تومان", "").replace("(مقطوع)", "").replace("توافقی", "NAN"),
                "size": None,
                "year": soup.find_all("span", class_="kt-group-row-item__value")[1].text if soup.find_all("span", class_="kt-group-row-item__value") else None, 
                "room_number": None, 
                "color": soup.find_all("span", class_="kt-group-row-item__value")[2].text if len(soup.find_all("span", class_="kt-group-row-item__value")) > 2 else None, 
                "mileage": soup.find_all("span", class_="kt-group-row-item__value")[0].text if soup.find_all("span", class_="kt-group-row-item__value") else None,
                "CPU": None,
                "RAM": None
               }
    return row_data


def refrigerator_freezer(soup):
    row_data = {"cat1": "home-kitchen", 
                "cat2": "appliance", 
                "cat3": "refrigerator-freezer", 
                "title": soup.find_all("div", class_="kt-page-title__title kt-page-title__title--responsive-sized")[0].text, 
                "description": soup.find_all("p", class_="kt-description-row__text kt-description-row__text--primary")[0].text, 
                "url": url, 
                "sub_title": soup.find_all("div", class_="kt-page-title__subtitle kt-page-title__subtitle--responsive-sized")[0].text,
                "price": soup.find_all("p", class_="kt-unexpandable-row__value")[-1].text.replace(" تومان", "").replace("(مقطوع)", "").replace("توافقی", "NAN"),
                "size": None,
                "year": soup.find_all("span", class_="kt-group-row-item__value")[1].text if soup.find_all("span", class_="kt-group-row-item__value") else None, 
                "room_number": None, 
                "color": soup.find_all("span", class_="kt-group-row-item__value")[2].text if len(soup.find_all("span", class_="kt-group-row-item__value")) > 2 else None, 
                "mileage": soup.find_all("span", class_="kt-group-row-item__value")[0].text if soup.find_all("span", class_="kt-group-row-item__value") else None,
                "CPU": None,
                "RAM": None
               }
    return row_data


def washing_machines(soup):
    row_data = {"cat1": "home-kitchen", 
                "cat2": "appliance", 
                "cat3": "washing-machines", 
                "title": soup.find_all("div", class_="kt-page-title__title kt-page-title__title--responsive-sized")[0].text, 
                "description": soup.find_all("p", class_="kt-description-row__text kt-description-row__text--primary")[0].text, 
                "url": url, 
                "sub_title": soup.find_all("div", class_="kt-page-title__subtitle kt-page-title__subtitle--responsive-sized")[0].text,
                "price": soup.find_all("p", class_="kt-unexpandable-row__value")[-1].text.replace(" تومان", "").replace("(مقطوع)", "").replace("توافقی", "NAN"),
                "size": None,
                "year": soup.find_all("span", class_="kt-group-row-item__value")[1].text if soup.find_all("span", class_="kt-group-row-item__value") else None, 
                "room_number": None, 
                "color": soup.find_all("span", class_="kt-group-row-item__value")[2].text if len(soup.find_all("span", class_="kt-group-row-item__value")) > 2 else None, 
                "mileage": soup.find_all("span", class_="kt-group-row-item__value")[0].text if soup.find_all("span", class_="kt-group-row-item__value") else None,
                "CPU": None,
                "RAM": None
               }
    return row_data


category_function = {
    "buy-apartment": buy_apartment, 
    "buy-office": buy_office, 
    "buy-villa": buy_villa,
    "buy-old-house": buy_old_house, 
    "car": car, 
    "mobile-phones": mobile_phones,
    "laptop-notebook-macbook": laptop_notebook_macbook, 
    "motorcycles": motorcycles,
    "refrigerator-freezer": refrigerator_freezer,
    "washing-machines": washing_machines
}

def extract_data(category, url):
    ua = UserAgent()
    header = {'User-Agent':str(ua.random)}
    page = requests.get(url, headers=header)
    soup = BeautifulSoup(page.text, 'html.parser')
    if page.status_code == 404:
        return 404
    debug_log.append(soup)
    return category_function[category](soup)

### Crawling

In [298]:
res = []
debug_log = []
for idx, category, url in tqdm(zip(range(len(url_df[0])), url_df[0], url_df[1])):
    if idx > 1200:
        time.sleep(np.random.randint(10,100)/100)
        result = extract_data(category, url)
        if result != 404:
            res.append(result)
        if idx % 40 == 0 and idx != 0:
            pd.json_normalize(res).to_csv(f"./../data/tmp/res_{idx}.csv")
            pd.json_normalize(res).to_json(f"./../data/tmp/res_{idx}.json")
            res = []
            print(idx)
        if idx == 1439:
            pd.json_normalize(res).to_csv(f"./../data/tmp/res_{idx}.csv")
            pd.json_normalize(res).to_json(f"./../data/tmp/res_{idx}.json")

1234it [00:34, 15.71it/s]  

1240


1281it [01:27,  2.44it/s]

1280


1321it [02:07,  1.08it/s]

...

1361it [03:09,  1.06it/s]

1360


1401it [03:51,  1.13it/s]

1400


1440it [04:32,  5.29it/s]


In [296]:
debug_log[-1]

[]